# 課題 - Human Resources
***

<img src="https://1.bp.blogspot.com/-LHowbq0ZoNY/VcMlVYhOVgI/AAAAAAAAwZ4/mgMtHMx5fcM/s800/fukidashi_taisyoku_woman.png" alt="退職を考えている人のイラスト" width="30%" height="30%"/>

In [1]:
# import basic apis
import sys

import scipy as sp
import numpy as np
import pandas as pd
import matplotlib
import sklearn

from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# pickle
from sklearn.externals import joblib
# 学習モデル
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
# 評価手法
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
#  画面描画
import ipywidgets
from ipywidgets import interact

print('Python Version: {}'.format(sys.version))
print('pandas Version: {}'.format(pd.__version__))
print('matplotlib Version: {}'.format(matplotlib.__version__))
print('Numpy Version: {}'.format(np.__version__))
print('sklearn Version: {}'.format(sklearn.__version__))
print('ipywidgets Version: {}'.format(ipywidgets.__version__))

Python Version: 3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 18:21:58) 
[GCC 7.2.0]
pandas Version: 0.22.0
matplotlib Version: 2.2.2
Numpy Version: 1.14.2
sklearn Version: 0.19.1
ipywidgets Version: 7.2.1


## OSEMN Pipeline
****

以下のOSEMN(awesomeと発音)の手法に則って行う

1. **O**btaining the data is the first approach in solving the problem.

2. **S**crubbing or cleaning the data is the next step. This includes data imputation of missing or invalid data and fixing column names.

3. **E**xploring the data will follow right after and allow further insight of what our dataset contains. Looking for any outliers or weird data. Understanding the relationship each explanatory variable has with the response variable resides here and we can do this with a correlation matrix. 

4. **M**odeling the data will give us our predictive power on whether an employee will leave. 

5. I**N**terpreting the data is last. With all the results and analysis of the data, what conclusion is made? What factors contributed most to employee turnover? What relationship of variables were found? 

# O: データを取り込む
***

<img src="https://3.bp.blogspot.com/-YtrWSqttYsQ/WM9XglY6dtI/AAAAAAABCrE/FKxvLU_Dllkg7PN1RV8xSys-7M86MS1vwCLcB/s800/bg_digital_pattern_green.jpg" alt="データのイラスト" width="30%" height="30%">

あらかじめ、取り決めておいたデータ仕様(./data/データ仕様書.xlsx)と今回提出いただいたCSVファイルのデータ形式に相違がないことを確認した。  
ここでは、ファイルの読み込みを行う。

In [115]:
# import Sample Data to learn models

# Read the analytics csv file and store our dataset into a dataframe called "df"
#df = pd.DataFrame.from_csv('../input/HR_comma_sep.csv', index_col=None)
index_column = 'index'
source_csv = './data/final_hr_analysis_train.csv'
df = pd.read_csv(source_csv, index_col=index_column)

source_csv_proba = './data/final_hr_analysis_test.csv'
df_proba = pd.read_csv(source_csv_test, index_col=index_column)

+ 仕様どおり、'index'列が一意であることが確認できた。正常なデータであると判断し、以降の処理を継続する。
+ 'index'列をindexとして採用し、データフレームへの読み込みを実施。

# S: データクレンジング 
***

<img src="https://4.bp.blogspot.com/-nwgj7Uh-ooI/WGnPaIeQD1I/AAAAAAABA6M/Y8TUclXA93Q5WTT81nd4DdJep5fV1H8ywCLcB/s800/room_living_clean.png" alt="ピカピカのリビングのイラスト" width="30%" height="30%">

データ分析を行うにあたって、事前にデータのクレンジング処理(欠損値への対応、カテゴリカル変数の対応など)をおこなう。  

In [116]:
df.shape

(10499, 10)

## 可読性を高める

In [118]:
# column名称のりネーム
columns = {
    'satisfaction_level': 'satisfaction',
    'last_evaluation': 'evaluation',
    'number_project': 'projectCount',
    'average_montly_hours': 'averageMonthlyHours',
    'time_spend_company': 'yearsAtCompany',
    'Work_accident': 'workAccident',
    'promotion_last_5years': 'promotion',
    'sales': 'department',
    'left': 'turnover'
}

df = df.rename(columns=columns)
df_proba = df_proba.rename(columns=columns)

## 欠損値への対応

In [119]:
# 欠損値の有無確認
df.isnull().any()

turnover               False
satisfaction           False
evaluation             False
projectCount           False
averageMonthlyHours    False
yearsAtCompany         False
workAccident           False
promotion              False
department             False
salary                 False
dtype: bool

In [120]:
# データの中身を確認
df.head()

,turnover,satisfaction,evaluation,projectCount,averageMonthlyHours,yearsAtCompany,workAccident,promotion,department,salary
index,,,,,,,,,,
10438,0,0.53,0.52,2,135,4,0,0,technical,medium
9236,0,0.77,0.53,5,256,3,0,0,accounting,medium
818,1,0.89,0.79,3,149,2,0,0,support,medium
11503,0,0.64,0.63,3,156,6,1,0,support,low
11721,0,0.98,0.74,4,151,3,0,0,sales,medium


## カテゴリカル変数への対応

In [121]:
# カテゴリカル変数を、one hot encoding
ohe_columns = ['department', 'salary']
X_dummies = pd.get_dummies(df, dummy_na=False, columns=ohe_columns)
print(X_dummies.dtypes)
# カテゴリカル変数を、one hot encoding(test)
X_dummies_proba = pd.get_dummies(df_proba, dummy_na=False, columns=ohe_columns)

turnover                    int64
satisfaction              float64
evaluation                float64
projectCount                int64
averageMonthlyHours         int64
yearsAtCompany              int64
workAccident                int64
promotion                   int64
department_IT               uint8
department_RandD            uint8
department_accounting       uint8
department_hr               uint8
department_management       uint8
department_marketing        uint8
department_product_mng      uint8
department_sales            uint8
department_support          uint8
department_technical        uint8
salary_high                 uint8
salary_low                  uint8
salary_medium               uint8
dtype: object


+ 欠損がないデータなので、欠損値の対応は不要であった。  
+ カラム名は、よりわかりやすい名称へと変更した。 
+ カテゴリカル変数の['department', 'salary']にたいしてOneHotEncodingを実施。

# E: データの調査
*** 

<img src="https://1.bp.blogspot.com/-0mU8U4WPRAs/WerKkBA4WQI/AAAAAAABHpI/_oa_Oxu7ThYhD0-14-Pe4etwp6jPX9jTACLcBGAs/s800/computer_hakui_doctor_man.png" alt="データ分析してる人のイラスト" width="30%" height="30%">

データの中身を探索し、特徴量などを決定。また、データに偏りがないかも調べておく必要がある。

## データ内の正負の割合・偏りを確認

In [122]:
# 導出すべき値に偏りがありすぎると、モデルのパフォーマンスに影響する
# その場合、何らかの対策(SMOTE、ダウンサンプリングなど)を行う必要あり
turnover_rate = df.turnover.value_counts() / len(df)
print(turnover_rate)

0    0.758739
1    0.241261
Name: turnover, dtype: float64


In [123]:
# Display the statistical overview of the employees
df.describe()

,turnover,satisfaction,evaluation,projectCount,averageMonthlyHours,yearsAtCompany,workAccident,promotion
count,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000
mean,0.241261,0.612631,0.717606,3.802553,201.121726,3.496428,0.145538,0.020764
std,0.427869,0.248155,0.170685,1.238923,49.834992,1.447783,0.352659,0.142600
min,0.000000,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000
25%,0.000000,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000
50%,0.000000,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000
75%,0.000000,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000


## 正負の特徴量の差異を比較

In [124]:
# Overview of summary (Turnover V.S. Non-turnover)
target_name = 'turnover'
turnover_Summary = df.groupby(target_name)
turnover_Summary.mean()

,satisfaction,evaluation,projectCount,averageMonthlyHours,yearsAtCompany,workAccident,promotion
turnover,,,,,,,
0,0.667201,0.716962,3.783204,198.930203,3.376601,0.175747,0.025734
1,0.441015,0.719633,3.863403,208.013818,3.873273,0.050533,0.005132


+ 'turnover'値がおおよそ、0:1 = 3:1 であった。均等ではないが、そこまで大きな問題でないと判断し、対応は行わない。

# M: モデル構築
***

<img src="https://2.bp.blogspot.com/-Eaqkz47FqEQ/WEztN1keMTI/AAAAAAABAUk/Kch-IzHmkQsMKRRauuRk3L95QhgewY7KwCLcB/s800/ai_study_kikaigakusyu.png" alt="機械学習のイラスト" width="30%" height="30%">

テスト・訓練データへ分割し、学習を実施  
複数モデルを構築・比較することで、よりよい学習モデルの採用が可能とする。

## テスト・訓練データの分割

In [125]:
# X,yを定義
X = X_dummies.drop(target_name, axis=1)

y = df[target_name]

# X,yを定義(testの方)
X_proba = X_dummies_proba.drop(target_name, axis=1)

y_proba = df_test[target_name]

In [126]:
# 訓練データ・テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=123, stratify=y)

X_train.head()

,satisfaction,evaluation,projectCount,averageMonthlyHours,yearsAtCompany,workAccident,promotion,department_IT,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_high,salary_low,salary_medium
index,,,,,,,,,,,,,,,,,,,,
14204,0.64,0.50,4,253,10,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
4086,0.16,0.84,3,238,6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
9971,0.49,0.74,2,154,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
14602,0.45,0.55,2,148,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5374,0.58,0.62,5,184,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


## パイプラインの構築

In [13]:
# 標準化も指定しておく
pipe_rfc = Pipeline([('scl', StandardScaler()),
                     ('est', RandomForestClassifier(random_state=1))])
pipe_gbc = Pipeline([('scl', StandardScaler()),
                     ('est', GradientBoostingClassifier(random_state=1))])
pipe_xgb = Pipeline([('scl', StandardScaler()), ('est', xgb.XGBClassifier())])
pipe_lgb = Pipeline([('scl', StandardScaler()), ('est', lgb.LGBMClassifier())])
# pipe_mlp = Pipeline([('scl',StandardScaler()),('est',MLPClassifier(hidden_layer_sizes=(5,2), max_iter=500, random_state=1))]) # hidden_layer_sizes を変更して、評価値の変化を見る
# パラメータいろいろ

# 使用する学習器の名前
pipe_names = ['RandomForest', 'GradientBoosting', 'XGBoost', 'LightGBM']
pipe_lines = [pipe_rfc, pipe_gbc, pipe_xgb, pipe_lgb]
print('## 使用する学習器\n' + ','.join(pipe_names))

## 使用する学習器
RandomForest,GradientBoosting,XGBoost,LightGBM


## ハイパーパラメータの設定

In [14]:
# パラメータグリッドの設定
#param_grid_logistic = {'est__C':[0.1,1.0,10.0,100.0], 'est__penalty':['l1','l2']}
param_rand_rfc = {
    'est__n_estimators': [1000],
    'est__criterion': ['gini', 'entropy'],
    'est__min_samples_leaf': sp.stats.randint(10, 15),
    'est__min_samples_split': sp.stats.randint(2, 10),
    'est__max_depth': sp.stats.randint(2, 5),
    'est__random_state': [1]
}
param_rand_gbc = {
    'est__n_estimators': [50, 100],
    'est__min_samples_leaf': sp.stats.randint(10, 15),
    'est__min_samples_split': sp.stats.randint(2, 10),
    'est__subsample': [0.8, 1.0]
}
param_rand_xgb = {
    'est__silent': [False],
    'est__max_depth': [6, 10, 15, 20],
    'est__learning_rate': [0.001, 0.01, 0.1, 0.2, 0, 3],
    'est__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'est__colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'est__colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'est__min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
    'est__gamma': [0, 0.25, 0.5, 1.0],
    'est__reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
    'est__n_estimators': [100]
}
param_rand_lgb = {
    'est__learning_rate': [0.01, 0.02],
    'est__n_estimators': [300, 400, 600, 800, 1000],
    # 'num_leaves':[4,8,16,32],
    'est__max_depth': [2, 3, 4, 5, 6],
    'est__boosting_type': ['gbdt'],
    #'est__objective': ['lambdarank'],
    'est__random_state': [1],
    # feature_fraction -> colsample_bytree
    # bagging_fraction -> subsample
    # bagging_freq -> subsample_freq
    'est__min_data_in_leaf': [10, 20],
    'est__scoring': ['ndcg'],
    # 'colsample_bytree' : [0.25,0.5,0.6,0.7,0.8],
    # 'colsample_bytree' : [0.6,0.7,0.8,0.9],
    'est__feature_fraction': [1, 0.9, 0.8, 0.4],
    'est__subsample': [1, 0.9, 0.8, 0.5],
    'est__max_bin': [50, 100, 200],
    'est__is_unbalance': [True, False],

    # 'min_child_weight':[5,10,25,50],
    # 'n_jobs': [3]
}

## 学習モデルの構築とファイルへの保存

In [15]:
# 学習
params = [param_rand_rfc, param_rand_gbc, param_rand_xgb, param_rand_lgb]
best_estimator = []
for pipe, param in zip(pipe_lines, params):
    print(
        '----------------------------------------------------------------------------------------------'
    )
    print('探索空間:%s' % param)
    rscv = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param,
        cv=10,
        n_iter=10,
        scoring='roc_auc',
        random_state=1,
        n_jobs=2)
    #gs = GridSearchCV(estimator=pipe, param_grid=param, scoring='f1', cv=3)
    rs = rscv.fit(X, y.as_matrix().ravel())
    # gs.best_estimator_でベストモデルを呼び出せる
    best_estimator.append(rs.best_estimator_)
    # gs.best_score_で上記ベストモデルのCV評価値（ここではf1スコア）を呼び出せる
    print('Best Score %.6f\n' % rs.best_score_)
    print('Best Model: %s' % rs.best_estimator_)

----------------------------------------------------------------------------------------------
探索空間:{'est__n_estimators': [1000], 'est__criterion': ['gini', 'entropy'], 'est__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fec507147b8>, 'est__min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fec50714908>, 'est__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fec50714a90>, 'est__random_state': [1]}
Best Score 0.976674

Best Model: Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=14, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=1

In [106]:
%%bash
# 前回出力されたファイルを削除
mkdir -p ./model
rm -f ./pkl/*

In [107]:
pickle_name = 'model.pkl'  # 出力する学習モデルファイル名
pickle_dir = './model/'

best_ests = []
print('\n## ファイルへ出力')
# 学習とファイル保存
for i, est in enumerate(best_estimator):
    # 学習モデルをファイルに保存する
    filename = pickle_dir + pipe_names[i] + '_' + pickle_name
    joblib.dump(est, filename)
    print(filename + ' が保存されました。')


## ファイルへ出力
./model/RandomForest_model.pkl が保存されました。
./model/GradientBoosting_model.pkl が保存されました。
./model/XGBoost_model.pkl が保存されました。
./model/LightGBM_model.pkl が保存されました。


In [108]:
%%bash
ls ./model

GradientBoosting_model.pkl
LightGBM_model.pkl
RandomForest_model.pkl
XGBoost_model.pkl


## 評価スコアの出力

In [128]:
# スコアとして使用する値
columns = ['正解率', '適合率', '再現率', 'F1スコア', 'AUC']
print('## 使用する評価スコア\n' + ','.join(columns))
df = pd.DataFrame(columns=columns)

# 学習モデルごとにスコアの算出
for (i, pipe) in enumerate(best_estimator):
    # それぞれの評価スコアの算出
    #print('%s: %.3f'%(pipe_names[i], accuracy_score(y_test.as_matrix().ravel(), pipe.predict(X_test))))
    acc = accuracy_score(y_test.as_matrix().ravel(), pipe.predict(X_test))
    #print('%s: %.3f'%(pipe_names[i], precision_score(y_test.as_matrix().ravel(), pipe.predict(X_test))))
    pre = precision_score(y_test.as_matrix().ravel(), pipe.predict(X_test))
    #print('%s: %.3f'%(pipe_names[i], recall_score(y_test.as_matrix().ravel(), pipe.predict(X_test))))
    rec = recall_score(y_test.as_matrix().ravel(), pipe.predict(X_test))
    #print('%s: %.3f'%(pipe_names[i], f1_score(y_test.as_matrix().ravel(), pipe.predict(X_test))))
    f1 = f1_score(y_test.as_matrix().ravel(), pipe.predict(X_test))
    #print('%s: %.3f'%(pipe_names[i], roc_auc_score(y_test.as_matrix().ravel(), pipe.predict(X_test))))
    auc = roc_auc_score(y_test.as_matrix().ravel(), pipe.predict(X_test))

    # DataFrameへ行の追加（評価スコアを渡す）
    df.loc[pipe_names[i]] = [acc, pre, rec, f1, auc]

## 使用する評価スコア
正解率,適合率,再現率,F1スコア,AUC


/home/yusukefulltype/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yusukefulltype/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yusukefulltype/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yusukefulltype/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The tr

In [129]:
display(df) # とりあえず一回出しとく

,正解率,適合率,再現率,F1スコア,AUC
RandomForest,0.911429,0.976261,0.648915,0.779621,0.821947
GradientBoosting,0.962857,0.953488,0.889546,0.920408,0.937868
XGBoost,0.964762,0.971678,0.879684,0.923395,0.935762
LightGBM,0.971429,0.951515,0.928994,0.940120,0.956964


## ファイル読み込みと確率の出力

In [131]:
# スコアをテーブルに出力(プルダウンで選択されたものを出力)
print('■選択されたメトリクスのスコア順(降順)で表示→ベストモデルで予測→CSVで出力')


@interact(Metrics=columns)
def draw_table(Metrics):
    df2 = df.sort_values(
        by=Metrics, ascending=False).loc[:, [Metrics]]  # ソートした列だけ抽出
    df2 = df2.rename(columns={Metrics: 'スコア'})
    display(df2)
    best_model_name = df2.iloc[0].name
    print('best score : ' + best_model_name)
    model_name = pickle_dir + best_model_name + '_' + pickle_name
    clf = joblib.load(model_name)
    print(model_name + ' が読み込まれました。')
    # 予測
    predict = clf.predict_proba(X_proba)
    # 予測値に元のindex付与して、dataframe化
    df_predict = pd.DataFrame(
        predict, columns=['proba_0', 'proba_1'], index=X_proba.index)
    # output csv
    csv_name = './' + best_model_name + '_proba_score.csv'
    df_predict.proba_1.to_csv(csv_name, index=True,header=True)
    print(csv_name + ' が出力されました。')

■選択されたメトリクスのスコア順(降順)で表示→ベストモデルで予測→CSVで出力


interactive(children=(Dropdown(description='Metrics', options=('正解率', '適合率', '再現率', 'F1スコア', 'AUC'), value='正解…

# N:結論・提案
***

<img src="https://4.bp.blogspot.com/-EjZ4ENmfIkc/V9PE9nu6eKI/AAAAAAAA9ko/I1hPkXoivi4WWdibdh2JQw1kgeVXwu0AgCLcB/s800/kjhou_seifuku.png" alt="機械学習のイラスト" width="30%" height="30%">

+ ここで力尽きた…  

本来は"E"のデータ調査の部分で、個々の特徴量を細かく分析しておく必要がある。  

今回はスコアのみで評価のため、深く掘り下げていないが、ビジネスでは「何が特徴量として結果に効いているのか？」を顧客は知りたがっている。  
そこを含めて、分析・予測結果から何ができるのかをプレゼンするまでが、お仕事。  
「いい予測結果がでました」だけでは弱い。  